In [1]:
import ulmo
#from ulmo import cdec
from ulmo.util import convert_datetime

import os
import shutil
import sys
import pandas as pd
import numpy as np
import plotly.express as px
import time
from datetime import datetime

In [27]:
outer_dir = os.path.abspath(os.path.join(os.getcwd() ,".."))
unimpath = os.path.join(outer_dir, "IFT_files", 'Unimpaired Flow')
# Check if the directory exists, if not, create it
if not os.path.exists(unimpath):
    os.makedirs(unimpath)
ref_dir = os.path.abspath(os.path.join(os.getcwd() ,"Reference Files"))
nav_div_csv = os.path.join(ref_dir, 'Indian','Navarro_estimated_diversions.csv')
# Current date as text in YYYYMMDD format
date = datetime.today().strftime('%Y%m%d')


LOI_name = "Sproul" #Sproul or Indian 
if LOI_name == "Sproul":
    site_code = '11476600' # Bull Creek gage https://waterdata.usgs.gov/nwis/inventory/?site_no=11476600&agency_cd=USGS
elif LOI_name == "Indian":
    site_code = '11468000' # Navarro Gage https://waterdata.usgs.gov/nwis/inventory/?site_no=11468000&agency_cd=USGS
else:
    print("LOI not recognized")

file_out = os.path.join(unimpath, LOI_name + "_" + date + ".csv")

In [16]:
def download_daily_gage_data(gage_id):
    """Function to download stream gage data from USGS API"""
    result = ulmo.usgs.nwis.get_site_data(site_code=gage_id, 
                                          service='dv',
                                          parameter_code='00060',
                                          methods='all',
                                          start = '1850-1-1',
                                          end = '2021-9-30'
                                         )
    if result == {}:
        result = ulmo.usgs.nwis.get_site_data(site_code=str(site_code), 
                                              service='dv',
                                              parameter_code='72137',
                                              methods='all',
                                              start = '1850-1-1',
                                              end = '2021-9-30'
                                             )
        if result == {}:
            print("discharge parameters not available for this gage")
            return

    first_key = list(result.keys())[0]
    # Convert to dataframe
    first_key = list(result.keys())[0]
    df = pd.DataFrame.from_dict(result[first_key]['values'])
    df["value"] = df["value"].replace(['---','BRT','ART','-1', 'nan'],np.NaN)
    df[['value']] = df[['value']].apply(pd.to_numeric)
    df['datetime'] =  pd.to_datetime(df['datetime'])
    df['gage_id'] =  gage_id
    return df

In [17]:
def evaluate_record(df):
    """Evaluate the gage data completeness"""
    first = df['datetime'].min()
    last = df['datetime'].max()
    Qmin = df['value'].min()
    Qmax = df['value'].max()
    delta = last - first
    all_records = len(df)
    df_valid = df.dropna(subset=['value'])
    valid_records = len(df_valid)
    pct_missing = all_records / valid_records
    pct_valid = valid_records / delta.days
    print('First date: {}'.format(first))
    print('Last date: {}'.format(last))
    print('Min Flow: {} cfs'.format(Qmin))
    print('Max Flow: {} cfs'.format(Qmax))
    print('Percent of records with missing data: {0:.1%}'.format(1 - pct_missing))
    print('Percent of the timeframe with valid records: {0:.1%}'.format(pct_valid))
    df_month = df_valid.resample('M', on='datetime').count()
    df_month = df_month['value'].reset_index()
    df_month.rename(columns={'value': 'valid_count'}, inplace=True)
    #Remove months with zero days
    df_month = df_month.loc[df_month.valid_count > 0]
    df_month['days_in_month'] = df_month['datetime'].dt.daysinmonth
    df_month['pct_valid_in_month'] = df_month['valid_count'] / df_month['days_in_month']
    pct_partial_month = len(df_month.loc[df_month['pct_valid_in_month'] < 1]) / len(df_month)
    print('Percent of partial months: {0:.1%}'.format(pct_partial_month))
    
    
    #Evaluation the number in days in each month
    df_day_freq = (df_month.groupby('valid_count')
               .agg({'days_in_month':'count'})
               .reset_index()
               .rename(columns={'days_in_month':'Count_months'})
              )
    print('Number of unique day counts per month: {}'.format(len(df_day_freq)))
    return df_day_freq

In [18]:
df = download_daily_gage_data(str(site_code))
df_eval = evaluate_record(df)

processing data from request: https://waterservices.usgs.gov/nwis/dv/?format=waterml&site=11476600&parameterCd=00060&startDT=1850-01-01&endDT=2021-09-30


First date: 1960-10-01 00:00:00
Last date: 2021-09-30 00:00:00
Min Flow: 0.02 cfs
Max Flow: 4900.0 cfs
Percent of records with missing data: 0.0%
Percent of the timeframe with valid records: 97.8%
Percent of partial months: 0.3%
Number of unique day counts per month: 5


In [19]:
# Add in month and day fields
df['Year'] = df.datetime.dt.year
df['Month'] = df.datetime.dt.month
df['Day'] = df.datetime.dt.day

#Add in water year
def assign_wy(row):
    if row.datetime.month>=10:
        return(pd.datetime(row.datetime.year+1,1,1).year)
    else:
        return(pd.datetime(row.datetime.year,1,1).year)

df['Water Year'] = df.apply(lambda x: assign_wy(x), axis=1)

E:\Users\kklausmeyer\AppData\Local\Temp\140\ipykernel_96752\344424719.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return(pd.datetime(row.datetime.year+1,1,1).year)
E:\Users\kklausmeyer\AppData\Local\Temp\140\ipykernel_96752\344424719.py:11: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return(pd.datetime(row.datetime.year,1,1).year)


In [20]:
# Check for complete years
df_year = (df.groupby(['gage_id', 'Water Year'])
           .agg({'value':'mean', 'datetime':'count'})
           .reset_index()
           .rename(columns={'value':'avg_ann_flow', 'datetime':'days_with_data'})
          )
df_year.sort_values(by=['days_with_data', 'Water Year'], ascending=[True, True])

,gage_id,Water Year,avg_ann_flow,days_with_data
58,11476600,2019,1.173333,30
59,11476600,2020,22.647731,216
0,11476600,1961,124.199452,365
1,11476600,1962,77.458904,365
2,11476600,1963,175.044932,365
...,...,...,...,...
39,11476600,2000,109.387978,366
43,11476600,2004,109.756721,366
47,11476600,2008,82.230246,366
51,11476600,2012,75.633115,366


In [21]:
# Calculate the unimpaired flow  
if LOI_name == "Sproul":
    # formula from "...instream-flow-management\Reference Files\SF_Sproul\Sproul Hydrographs fr Bull Creek v1.xlsm"
    # Documentation from California Trout and Humboldt State University 2018 Sproul Creek Instream Flow Study -- Final Report, prepared for the North Coast Regional Water Quality Control Board and State Water Resources Control Board, Arcata, California. 
    df['flow'] = (np.where(df['value']<2.706, 0.218*df['value']**2-0.0144*df['value'], df['value']*0.739))/24 * 5
    df['flow'] = np.where(df['flow']<0, 0, df['flow'])
elif LOI_name == "Indian":
    # values from "...instream-flow-management\Reference Files\Indian\Navarro_to_Indian_Creek_Flow.xlsx""
    DApod = 39 #sqmiles
    DAgage = 303
    Ppod = 40.4 # inches from PRISM
    Pgage = 40
    factor = (DApod/ DAgage) * (Ppod/ Pgage)
    df_nav = pd.read_csv(nav_div_csv)
    df_nav = df_nav.drop(['Month'], axis=1)
    df1 = pd.merge(df, df_nav[['Month.1', 'Daily cfs']], left_on='Month', right_on="Month.1", how='left')
    df1["nav_unimpaired"] = df1['value'] + df1['Daily cfs']
    #Calc Indian Creek unimpaired
    df1['flow'] = df1["nav_unimpaired"] * factor
    df = df1.drop(['Month.1', "nav_unimpaired", 'Daily cfs'], axis=1)
    

else:
    print("LOI not recognized")





In [22]:
df

,value,qualifiers,datetime,gage_id,Year,Month,Day,Water Year,flow
0,4.50,A,1960-10-01,11476600,1960,10,1,1961,0.692813
1,4.50,A,1960-10-02,11476600,1960,10,2,1961,0.692813
2,4.50,A,1960-10-03,11476600,1960,10,3,1961,0.692813
3,4.50,A,1960-10-04,11476600,1960,10,4,1961,0.692813
4,4.50,A,1960-10-05,11476600,1960,10,5,1961,0.692813
...,...,...,...,...,...,...,...,...,...
21790,0.07,A,2021-09-26,11476600,2021,9,26,2021,0.000013
21791,0.10,A,2021-09-27,11476600,2021,9,27,2021,0.000154
21792,0.12,A,2021-09-28,11476600,2021,9,28,2021,0.000294
21793,0.12,A,2021-09-29,11476600,2021,9,29,2021,0.000294


In [30]:
# Export data
df.rename(columns={'datetime': 'DTTM'}, inplace=True)
df.drop(['qualifiers', 'value', 'gage_id'], axis=1, inplace=True)
df.to_csv(file_out, encoding='utf-8', index=False)